In [1]:
import argparse
import torch
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from pathlib import Path
import json
resume = Path('saved/models/ChatbotLSTM/30epoch/model_best.pth')
cfg_fname = resume.parent / 'config.json'
config = ConfigParser(json.load(open(cfg_fname, 'r')))
config.resume = resume

In [2]:
logger = config.get_logger('inference')

# setup data_loader instances
data_loader = config.init_obj(
    'inference_data_loader',
    module_data,
    text_field_path=config.resume.parent / 'TEXT.Field',
    vocab_path=config.resume.parent / 'TEXT.Vocab'
)
logger.info('Load data loader')

# build model architecture
encoder = config.init_obj(
    'encoder_arch', module_arch,
    vocab_size=data_loader.vocab_size,
    padding_idx=data_loader.padding_idx,
    hidden_size=config['hidden_size'],
    embed_size=config['embed_size']
)
encoder.eval()
logger.info(encoder)
decoder = config.init_obj(
    'decoder_arch', module_arch,
    embedding=encoder.embedding,
    embed_size=config['embed_size'],
    hidden_size=config['hidden_size'],
    vocab_size=data_loader.vocab_size
)
decoder.eval()
logger.info(decoder)
model_idx = dict([('encoder', 0), ('decoder', 1)])
models = [encoder, decoder]

logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume, map_location=torch.device('cpu'))
for idx in range(len(models)):
    models[idx].load_state_dict(
        checkpoint['{}_state_dict'.format(type(models[idx]).__name__)]
    )

greedy_decoder = config.init_obj(
    'inference_arch', module_arch,
    encoder=encoder,
    decoder=decoder,
    init_idx=data_loader.init_idx
)
logger.info(greedy_decoder)

Load data loader
ChatbotEncoder(
  (embedding): Embedding(12055, 50, padding_idx=1)
  (lstm): LSTM(50, 512, num_layers=2, dropout=0.1, bidirectional=True)
)
Trainable parameters: 9212542
LuongAttnDecoderRNN(
  (embedding): Embedding(12055, 50, padding_idx=1)
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (lstm): LSTM(50, 512, num_layers=2, dropout=0.1)
  (concat): Linear(in_features=1024, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=12055, bias=True)
  (attn): Attention(
    (attn): Linear(in_features=1024, out_features=512, bias=True)
  )
)
Trainable parameters: 11093397
Loading checkpoint: saved/models/ChatbotLSTM/30epoch/model_best.pth ...
GreedySearchDecoder(
  (encoder): ChatbotEncoder(
    (embedding): Embedding(12055, 50, padding_idx=1)
    (lstm): LSTM(50, 512, num_layers=2, dropout=0.1, bidirectional=True)
  )
  (decoder): LuongAttnDecoderRNN(
    (embedding): Embedding(12055, 50, padding_idx=1)
    (embedding_dropout): Dropout(p=0.1, inpl

In [ ]:
with torch.no_grad():
    text = input("Input text: ")
    x, x_len = data_loader.preprocess(text)
    # print(x)
    talk_seq = torch.ones_like(x)
    talk_seq[0:-1], talk_seq_len = x[1:], x_len - 1
    # x, x_len = x.to(device), x_len.to(device)
    all_tokens, all_scores = greedy_decoder(talk_seq, talk_seq_len, data_loader.sent_len)
    converted_text = data_loader.convert_ids_to_text(all_tokens)

Input text:  can i help you


In [1]:
import re

In [2]:
l = []
t = ['hello', '12', '1', ',', 'can', 'r13', ':']
for tok in t:
    i = re.sub(r'[\W]', '', tok)
    if i:
        l += [i]
l

['hello', '12', '1', 'can', 'r13']